In [2]:
library(igraph)
#library(data.table)
#library(network)
#library(intergraph)

In [3]:
version

               _                           
platform       x86_64-apple-darwin15.6.0   
arch           x86_64                      
os             darwin15.6.0                
system         x86_64, darwin15.6.0        
status                                     
major          3                           
minor          4.4                         
year           2018                        
month          03                          
day            15                          
svn rev        74408                       
language       R                           
version.string R version 3.4.4 (2018-03-15)
nickname       Someone to Lean On          

### Q12
features: top 5 pageranks of the actors in each movie

model: linear regression

In [4]:
load("Q12data.RData")
# g <- the graph of movies in part 2
# communities <- the communities result in part 2
# actorId2pagerank <- actor's pagerank, e.g. pagerank of actor 0 = actorId2pagerank[1]
# movieId2actorIds <- movies' actor list, e.g. actor list of movie 0 = movieId2actorIds[[1]]

In [5]:
get_feature_pageranks <- function(movieId) {
    actors <- movieId2actorIds[[movieId+1]]
    pagerank <- c()
    for (actorId in actors) {
        pagerank <- c(pagerank, actorId2pagerank[actorId+1])
    }
    return(sort(pagerank, decreasing=TRUE)[1:5])
}

df <- data.frame(matrix(ncol = 6, nrow = 0))
colnames(df) <- c("rating","pagerank1","pagerank2","pagerank3","pagerank4","pagerank5")

for (i in 1:vcount(g)){
    if (is.na(V(g)$rating[i])) { next }
    
    movie_id <- as.numeric(V(g)$name[i])
    movie_rating <- V(g)$rating[i]
    df[nrow(df)+1, ] <- c(movie_rating, get_feature_pageranks(movie_id))
}

mod <- lm(rating ~ ., data = df)
summary(mod)


Call:
lm(formula = rating ~ ., data = df)

Residuals:
    Min      1Q  Median      3Q     Max 
-5.9464 -0.7631  0.1215  0.8752  3.8247 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept)  5.910e+00  8.017e-03 737.146  < 2e-16 ***
pagerank1   -8.560e+02  2.570e+02  -3.331 0.000866 ***
pagerank2   -4.416e+02  7.751e+02  -0.570 0.568892    
pagerank3    1.479e+03  1.493e+03   0.990 0.322033    
pagerank4    3.858e+03  2.340e+03   1.649 0.099170 .  
pagerank5    1.155e+04  2.152e+03   5.366 8.07e-08 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 1.244 on 133257 degrees of freedom
  (56 observations deleted due to missingness)
Multiple R-squared:  0.003585,	Adjusted R-squared:  0.003548 
F-statistic:  95.9 on 5 and 133257 DF,  p-value: < 2.2e-16


In [6]:
RSS <- c(crossprod(mod$residuals))
MSE <- RSS / length(mod$residuals)
RMSE <- sqrt(MSE)
cat("RMSE:",RMSE)

RMSE: 1.243792

In [7]:
movie_id_targets <- c(10321, 39182, 78995)
movie_name_targets <- c("Batman v Superman: Dawn of Justice (2016)", "Mission: Impossible - Rogue Nation (2015)", "Minions (2015)")

In [8]:
for (i in 1:3) {
    movie_id <- movie_id_targets[i]
    cat(movie_name_targets[i], '\n')
    
    node_id <- which(V(g)$name == movie_id)
    rating_truth <- V(g)$rating[node_id]
    cat("Ground truth rating:", rating_truth, '\n')
    
    features <- data.frame(matrix(ncol = 5, nrow = 1))
    colnames(features) <- c("pagerank1","pagerank2","pagerank3","pagerank4","pagerank5")
    features[1, ] <- get_feature_pageranks(movie_id)
    rating_prediction <- predict(mod, features)
    cat("Predicted rating:", rating_prediction, '\n', '\n')
}

Batman v Superman: Dawn of Justice (2016) 
Ground truth rating: NA 
Predicted rating: 6.303812 
 
Mission: Impossible - Rogue Nation (2015) 
Ground truth rating: NA 
Predicted rating: 6.124734 
 
Minions (2015) 
Ground truth rating: NA 
Predicted rating: 6.081247 
 
